In [1]:
using Oscar
using Combinatorics
pm = Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.11.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The OSCAR Development Team


Polymake

In [16]:
include("matroid_realization.jl");
include("reduceIdeals.jl")
include("d3n10_special_matroids.jl")

d3n10 = vec(readlines("d3n10.dat"));


#remove monomial factors
function clean(f,R)
    
    if !(f == 0)
    
        if length(f) == 1
            
            return R(coeff(f,1))
            
        else
            
            fFactors = factor(f)
    
            FactorsDict = Dict(fFactors)
    
            cleanf = prod([k^(FactorsDict[k]) for k in keys(FactorsDict) if (length(k)>1||is_constant(k))])
        
            return cleanf
        
        end
        
    else
        
        return f
        
    end
    
end
    

function matroid_to_reduced_expression(Q, F,k)
    
    charts = [c for c in circuits(Q) if length(c) == rank(Q)+1]
    A = charts[1]
    RQ = matroid_realization_space(Q, A, F)
    R = parent(RQ[1][1])
    Sgens = [s for s in RQ[2] if length(s) <= k]#new 13.1.2023
    I = reduce_ideal_full(RQ[1], Sgens, R, gens(R), false)
    
    
 
    if I isa String
        
        return I
        
     #I[1] = ideal generators, I[2] = subgroup generators   
    else
        
       Iclean = unique!([clean(f,R) for f in I[1]])
    
            
       return (Iclean, I[2])
        
       # return(I[1],I[2])
        
    end
end




matroid_to_reduced_expression (generic function with 1 method)

In [3]:
d3n10

5249-element Vector{String}:
 "*******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "****************************0**"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "******************************0"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*********************0*********"
 "0******0***********************" ⋯ 58 bytes ⋯ "********************

In [4]:
has_property #matroids with "3 lines property"

151-element Vector{Any}:
  324
  423
  428
  450
  630
  635
  798
  884
  894
  903
  904
  906
  908
    ⋮
 4810
 4822
 4825
 4829
 4839
 4841
 4903
 4910
 4916
 4917
 4919
 5085

In [6]:
#for an element n, counts how many lines its in
function in_lines(n,L)
    xn = 0
    for z in 1:length(L)
        if n in L[z]
           xn += 1 
        end
    end
    return xn
end

in_lines (generic function with 1 method)

In [10]:
#find simple matroids such that each element is contained in at least 3 lines
#this could be a function that depends on M and k, where k is the number of "lines"
has_property = []
for z in 1:length(d3n10)
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    L = cyclic_flats(Mz,2)
    xm = 0
    for t in 1:10
        if in_lines(t,L)>2
            xm +=1
        end
    end
    #print(z,"\n", xm ,"\n",find_lines(z),"\n")
    if xm == 10
        push!(has_property,z)
    end
end

LoadError: UndefVarError: in_lines not defined

In [11]:
#realized i should check that everything is connected
for z in has_property
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    if !(n_connected_components(Mz) == 1)
        print(z,"\n")
        
    end
    
end

In [5]:
function f_needs_more(I)
    
    cc = []
    
    for f in I
        
        g = [x for x in vars(f) if degree(f,x) == 1]
        
        if length(g)>0
            
            push!(cc,f)
            
        end
        
    end
        
        if length(cc)>0
            
            return "more work"
            
        else
            
            return "fully simplified"
            
        end
        
    end
        
            


f_needs_more (generic function with 1 method)

In [6]:
function int_gens(I)
    
    ints = [x for x in I if is_constant(x)]
    
    if length(ints)>1
        
        return true
        
    else
        
        return false
        
    end
    
end

int_gens (generic function with 1 method)

In [23]:
#classification

not_realizable = []

int_generator = []#algebraic closure of integral generator not equal to 1

need_more_not_principle = []#ideal not fullysimplified, not principle
need_more_principle = [] #ideals that simplifies to something principle, but can be further reduced

simplified_not_principle = []#ideal simplifies,al not principle
simplified_principle_n_reducible = [] #ideal simplifies to something principle
simplified_principle_reducible = []#ideal fully simplifies, principally generated in one variable

zero_ideal = []#ideal fully simplifies to 0

for z in has_property
    
    #println(z,"\n\n")
  
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,2)
    
    #println("ch1")
    if I[1] == [0] #reduces to 0
        
        push!(zero_ideal,z)
    
    elseif (1 in I[1] || -1 in I[1] || I == "Not Realizable 0 in Semigroup")
        
    #println("ch2")
        
        push!(not_realizable,z)
        
    elseif int_gens(I[1])
        
               
    #println("ch3")
        
        push!(int_generator,z)
        
    elseif f_needs_more(I[1]) == "more work"#not fully reduced
        
               
    #println("ch4")
        
        if length(I[1]) == 2
            
            push!(need_more_principle,z)
            
        else
        
            push!(need_more_not_principle,z)
            
        end
        
    elseif f_needs_more(I[1]) == "fully simplified"#fully reduced
        
               
   # println("ch5")
        
        if length(I[1]) == 2
            
            if length(vars(I[1][2])) == 1
                
                push!(simplified_principle_reducible,z)
                
            else
                
                push!(simplified_principle_n_reducible,z)
            end
            
        else
        
            push!(simplified_not_principle,z)
            
        end
        
    end
end


In [ ]:
8-element Vector{Vector{Any}}:
 [1072, 1546, 2671, 2742, 3123, 3599, 3658, 3666, 3725, 4825]
 [1607, 3648, 3802, 3840, 4493, 4789, 4800, 4808, 4809, 4810, 4839, 4841, 4910, 4917, 4919]
 [635, 1412, 2669, 3592]
 [630, 1080, 1417, 1438, 1544, 1558, 2460, 2743, 3156, 3417, 3419, 3426, 3626, 3634, 3770, 3801, 4822]
 [1521, 1522, 1543, 1556, 1593, 1602, 3588, 3645, 3647, 4568]
 [324, 423, 428, 450, 884, 894, 903, 904, 906, 908  …  1457, 2934, 3121, 3124, 3154, 3157, 3234, 3235, 3349, 3359]
 [798, 965, 995, 1524, 1594, 1596, 1598, 1674, 1675, 2740  …  3809, 4276, 4490, 4557, 4570, 4571, 4758, 4761, 4903, 5085]
 [1082, 1525, 1672, 2459, 2739, 2786, 2955, 2958, 3352, 3403, 3726, 3771, 3800, 3811, 3812, 3839, 4279, 4784, 4829, 4916]

In [24]:
C = [not_realizable,
int_generator,

need_more_not_principle,
need_more_principle,

simplified_not_principle,
simplified_principle_n_reducible,
simplified_principle_reducible,

zero_ideal]

8-element Vector{Vector{Any}}:
 [1072, 1544, 1546, 2671, 2742, 3123, 3599, 3626, 3658, 3666, 3725, 4825]
 [1607, 3648, 3802, 3840, 4493, 4789, 4800, 4808, 4809, 4810, 4839, 4841, 4910, 4917, 4919]
 [1412, 2669]
 [630, 1080, 1417, 1438, 1558, 2460, 2743, 3417, 3419, 3426, 3634, 3770, 3801, 4822]
 [1521, 1522, 1543, 1556, 1593, 1602, 3588, 3645, 3647, 4568]
 [324, 423, 428, 450, 884, 894, 903, 904, 906, 908  …  1457, 2934, 3121, 3124, 3154, 3157, 3234, 3235, 3349, 3359]
 [635, 798, 965, 995, 1524, 1594, 1596, 1598, 1674, 1675  …  3809, 4276, 4490, 4557, 4570, 4571, 4758, 4761, 4903, 5085]
 [1082, 1525, 1672, 2459, 2739, 2786, 2955, 2958, 3156, 3352  …  3726, 3771, 3800, 3811, 3812, 3839, 4279, 4784, 4829, 4916]

In [10]:
for c in C
    print(length(c),"\n")
end
lc = [length(c) for c in C]
sum(lc)

10
15
4
17
10
45
30
20


151

In [ ]:
12
15
4
11
10
45
30
24


In [12]:
#sort principle ideals that need more work, eliminate the ones with generator of form x-d, d

to_0_ns_principle = []
more_ns_principle = []

for z in C[4]
    
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,2)
    f = I[1][2]
    
    if is_univariate(f)
            
            var = vars(f)[1]
            
            if degree(f,var) == 1
                
                push!(to_0_ns_principle,z)
                
            end
            
    else
        
        push!(more_ns_principle,z)
        
    end
    
end
    
    

In [13]:
length(to_0_ns_principle) +
length(more_ns_principle)

17

In [9]:
for z in need_more_principle
    
     Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
     I = matroid_to_reduced_expression(Mz, ZZ,2)
    
    print("Matroid : ",z,"\n\n")
    
    if I isa String

        print(I,"\n\n")
        
    else
        
    for j in 1:length(I[1])
        print(j," Factored : ", factor(I[1][j]),"\n", "Expanded : ",I[1][j],"\n\n")
    end
    end
end

Matroid : 630

1 Factored : 0
Expanded : 0

2 Factored : 1 * (x_{1, 6} - 1) * (2*x_{2, 5} - 1)
Expanded : 2*x_{2, 5}*x_{1, 6} - 2*x_{2, 5} - x_{1, 6} + 1

Matroid : 1080

1 Factored : 0
Expanded : 0

2 Factored : 1 * (x_{1, 4}*x_{2, 5} - x_{1, 4}*x_{2, 6} - x_{2, 5}*x_{2, 6} + 2*x_{2, 6} - 1) * (x_{1, 4} - x_{2, 5})
Expanded : x_{1, 4}^2*x_{2, 5} - x_{1, 4}^2*x_{2, 6} - x_{1, 4}*x_{2, 5}^2 + 2*x_{1, 4}*x_{2, 6} - x_{1, 4} + x_{2, 5}^2*x_{2, 6} - 2*x_{2, 5}*x_{2, 6} + x_{2, 5}

Matroid : 1417

1 Factored : 0
Expanded : 0

2 Factored : 1 * (x_{1, 6}^2 + x_{1, 6}*x_{2, 6} - x_{1, 6} - 2*x_{2, 6} + 1)
Expanded : x_{1, 6}^2 + x_{1, 6}*x_{2, 6} - x_{1, 6} - 2*x_{2, 6} + 1

Matroid : 1438

1 Factored : 0
Expanded : 0

2 Factored : 1 * (x_{1, 5}^2 + 2*x_{1, 5}*x_{2, 6} - 2*x_{1, 5} - x_{2, 6})
Expanded : x_{1, 5}^2 + 2*x_{1, 5}*x_{2, 6} - 2*x_{1, 5} - x_{2, 6}

Matroid : 1558

1 Factored : 0
Expanded : 0

2 Factored : 1 * (2*x_{1, 6}*x_{2, 6} - x_{1, 6} - x_{2, 6}^2)
Expanded : 2*x_{1, 6}*x_{2

In [28]:
Mz = matroid_from_revlex_basis_encoding(d3n10[635], 3, 10)
I = matroid_to_reduced_expression(Mz, ZZ,2)

(fmpz_mpoly[0, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6}^2 - x_{2, 5} + x_{1, 6}, x_{2, 5}^2 - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1], fmpz_mpoly[1, -1, -x_{2, 5}*x_{1, 6} + x_{1, 6}^2, -x_{2, 5}, x_{2, 5}*x_{1, 6} - x_{1, 6}^2, x_{2, 5} - x_{1, 6}^2, -x_{2, 5}^2*x_{1, 6} + x_{2, 5}*x_{1, 6}^2, x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + x_{1, 6} - 1, x_{2, 5} - 1, x_{2, 5} - x_{1, 6}  …  x_{2, 5}*x_{1, 6} - x_{1, 6}, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6}^2, -x_{2, 5}*x_{1, 6} + x_{1, 6}^2 - x_{1, 6} + 1, x_{1, 6}, -x_{2, 5}*x_{1, 6}, -x_{2, 5} + 1, x_{1, 6} - 1, -x_{2, 5}*x_{1, 6} + x_{1, 6}, -x_{2, 5}^2*x_{1, 6} + x_{2, 5}*x_{1, 6}^2 + x_{2, 5}*x_{1, 6} - x_{1, 6}^2, -x_{1, 6} + 1])

In [29]:
#check if coefficients are in semigroup for ideals that still need more work

for z in union(C[3],C[4])
    print(z," : ","\n")
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,2)
    R = parent(I[1])
    
    notz = [f for f in I[1] if !(f == 0)]
    for f in notz
        
        print("generator : ",f,"\n\n")
        t = [x for x in vars(f) if degree(f,x) == 1]
         
        if length(t)>0
            v = t[1]   
            coefft = coefficient_v(v,f,R)
            print("variable : ",t,"\n","coefficient :",coefft,"\n","coeff in semigrouop :",coefft in I[2],"\n","would substitute :
                ","\n\n")
        end
    end
    print("\n\n\n")
end
#observations: for everything that doesn't reduce in an obvious way in C[4], the coefficient was not in the 
#semigroup somehow due to the simplification process. I can see how this can happen, but I don't know what it
#means for the math. 

#in the to_0_ns_principle, the integer coefficients not equal to 1 are not part of the 
#semigroup. In some cases the coefficient is 1 and that is in the semigroup, but its not getting rid of those. 
#When switching to QQ we see some become non realizable or simplify to 0.

#in C[3] each ideal had only one generator in a variable that could be solved for. In 3 cases, 
#the coefficient wasn't in the ideal.  In the case of 635 and 3592 the coefficient is in the semigroup, 
#but the simplification didn't get to the end. 

#635 and 3592 are the most interesting, but the reason why the ones with coefficient 1 above aren't reducing 
#to 0 may have something to do with it. it seems like substition will simplify  635 and 3592 down to a principle 
#ideal with univariate generator in degree >1, which will be disconnected.

#for the ones whose coefficient is not in the semigroup, try substituting variables anyways. Need something that 
#does that without depending on the semigroup. Won't be bad

1412 : 
generator : x_{1, 5}*x_{1, 6} - x_{1, 5}*x_{2, 6}^2 - x_{1, 6}^2 + 2*x_{1, 6}*x_{2, 6} - x_{1, 6}

variable : fmpz_mpoly[x_{1, 5}]
coefficient :x_{1, 6} - x_{2, 6}^2
coeff in semigrouop :false
would substitute :
                

generator : x_{1, 5}*x_{1, 6}^2 - x_{1, 5}*x_{1, 6} - 2*x_{1, 6}*x_{2, 6} + x_{1, 6} + x_{2, 6}^2

variable : fmpz_mpoly[x_{1, 5}]
coefficient :x_{1, 6}^2 - x_{1, 6}
coeff in semigrouop :false
would substitute :
                




2669 : 
generator : x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1

variable : fmpz_mpoly[x_{1, 6}]
coefficient :x_{2, 5}^2 - x_{2, 5} + 1
coeff in semigrouop :false
would substitute :
                

generator : x_{2, 5}^3*x_{1, 6} - x_{2, 5}^3 - x_{2, 5}^2*x_{1, 6}^2 - x_{2, 5}^2*x_{1, 6} + x_{2, 5}^2 + 2*x_{2, 5}*x_{1, 6}^2 - 2*x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + 2*x_{1, 6} - 1




630 : 
generator : 2*x_{2, 5}*x_{1, 6} - 2*x_{2, 5} - x_{1, 6} + 1

variable : fmpz_mpoly[x_{2, 5}, x_{1, 6}]
coefficient :2*x_{1, 6} 

In [20]:
for z in need_more_not_principle
    
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, QQ,2)
    
    if I isa String 
        
        print(z, ":", "\n\n", I,"\n\n\n")
        
    else
    
        R = parent(I[1][1])
        Sgens = [s for s in I[2] if length(s) <= 2]#new 13.1.2023
        G = reduce_ideal_full(I[1], Sgens, R, gens(R), false)
        
        if G isa String
        
            print(z," : ","\n\n","original : ", I[1], "\n\n","second : " ,G,"\n\n\n")
        
        else
        
            print(z," : ","\n\n","original : ", I[1], "\n\n","second : ",G[1],"\n\n\n")
            
        end
        
    end
    
end

1412 : 

original : fmpq_mpoly[0, x_{1, 5}*x_{1, 6} - x_{1, 5}*x_{2, 6}^2 - x_{1, 6}^2 + 2*x_{1, 6}*x_{2, 6} - x_{1, 6}, x_{1, 5}*x_{1, 6}^2 - x_{1, 5}*x_{1, 6} - 2*x_{1, 6}*x_{2, 6} + x_{1, 6} + x_{2, 6}^2]

second : fmpq_mpoly[0, x_{1, 5}*x_{1, 6} - x_{1, 5}*x_{2, 6}^2 - x_{1, 6}^2 + 2*x_{1, 6}*x_{2, 6} - x_{1, 6}, x_{1, 5}*x_{1, 6}^2 - x_{1, 5}*x_{1, 6} - 2*x_{1, 6}*x_{2, 6} + x_{1, 6} + x_{2, 6}^2]


2669 : 

original : fmpq_mpoly[0, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1, x_{2, 5}^3*x_{1, 6} - x_{2, 5}^3 - x_{2, 5}^2*x_{1, 6}^2 - x_{2, 5}^2*x_{1, 6} + x_{2, 5}^2 + 2*x_{2, 5}*x_{1, 6}^2 - 2*x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + 2*x_{1, 6} - 1]

second : fmpq_mpoly[0, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1, x_{2, 5}^3*x_{1, 6} - x_{2, 5}^3 - x_{2, 5}^2*x_{1, 6}^2 - x_{2, 5}^2*x_{1, 6} + x_{2, 5}^2 + 2*x_{2, 5}*x_{1, 6}^2 - 2*x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + 2*x_{1, 6} - 1]




In [12]:
for z in need_more_not_principle
    
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,2)
    
    if I isa String 
        
        print(z, ":", "\n\n", I,"\n\n\n")
        
    else
    
        R = parent(I[1][1])
        Sgens = [s for s in I[2] if length(s) <= 2]#new 13.1.2023
        G = reduce_ideal_full(I[1], Sgens, R, gens(R), false)
        
        if G isa String
        
            print(z," : ","\n\n","original : ", I[1], "\n\n","second : " ,G,"\n\n\n")
        
        else
        
            print(z," : ","\n\n","original : ", I[1], "\n\n","second : ",G[1],"\n\n\n")
            
        end
        
    end
    
end

1412 : 

original : fmpz_mpoly[0, x_{1, 5}*x_{1, 6} - x_{1, 5}*x_{2, 6}^2 - x_{1, 6}^2 + 2*x_{1, 6}*x_{2, 6} - x_{1, 6}, x_{1, 5}*x_{1, 6}^2 - x_{1, 5}*x_{1, 6} - 2*x_{1, 6}*x_{2, 6} + x_{1, 6} + x_{2, 6}^2]

second : fmpz_mpoly[0, x_{1, 5}*x_{1, 6} - x_{1, 5}*x_{2, 6}^2 - x_{1, 6}^2 + 2*x_{1, 6}*x_{2, 6} - x_{1, 6}, x_{1, 5}*x_{1, 6}^2 - x_{1, 5}*x_{1, 6} - 2*x_{1, 6}*x_{2, 6} + x_{1, 6} + x_{2, 6}^2]


2669 : 

original : fmpz_mpoly[0, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1, x_{2, 5}^3*x_{1, 6} - x_{2, 5}^3 - x_{2, 5}^2*x_{1, 6}^2 - x_{2, 5}^2*x_{1, 6} + x_{2, 5}^2 + 2*x_{2, 5}*x_{1, 6}^2 - 2*x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + 2*x_{1, 6} - 1]

second : fmpz_mpoly[0, x_{2, 5}^2*x_{1, 6} - x_{2, 5}*x_{1, 6} + x_{1, 6} - 1, x_{2, 5}^3*x_{1, 6} - x_{2, 5}^3 - x_{2, 5}^2*x_{1, 6}^2 - x_{2, 5}^2*x_{1, 6} + x_{2, 5}^2 + 2*x_{2, 5}*x_{1, 6}^2 - 2*x_{2, 5}*x_{1, 6} - x_{1, 6}^2 + 2*x_{1, 6} - 1]




In [30]:
Mz = matroid_from_revlex_basis_encoding(d3n10[4822], 3, 10)
I = matroid_to_reduced_expression(Mz, ZZ,2)
    


(fmpz_mpoly[0, 3*x_{1, 6} + 1], fmpz_mpoly[1, -1, x_{1, 6}, x_{1, 6} + 1, -x_{1, 6}, -x_{1, 6} - 1, -2, x_{1, 6} - 1, 2*x_{1, 6}, 2])

In [ ]:
R = parent(I[1][1])
Sgens = [s for s in I[2] if length(s) <= 2]#new 13.1.2023
G = reduce_ideal_full(I[1], Sgens, R, gens(R), false)